# Otimização para o Problema de Recobrimento de Conjuntos

In [1]:
from ORLibrary_SPC.fetch_instance import get_data
from ORLibrary_SPC.parse_instance import parse_instance

import cplex
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# !pip3 install PyQt5

## Modelo sem Relaxações

In [2]:
def modelo_MIP(instance_name: str, display_output=True):
    """
    Modelo usando CPLEX para obter uma solução ótima 
    para problema de recobrimento de conjuntos.

    Recebe uma instância no formato encontrado na ORLib e
    implementa uma solução gulosa sem relaxações.
    """

    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize)

    # Obtém e processa os dados da instância
    # m = # linhas
    # n = # colunas
    # C = lista de custos para cada coluna
    # J = conjunto de comprimento m de listas com as colunas que cobrem cada linha (J[i])

    get_data(instance_url=f'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/{instance_name}.txt')
    m, n, C, J = parse_instance(instance_name=instance_name)

    if not m:
        print('Instance parsing failed.')
        return

    # Função objetivo, minimizar os custos (C)
    cpx.variables.add(obj=C,
                        lb=[0] * n, ub=[1] * n,
                        types=[cpx.variables.type.binary] * n,
                        names=[f'x_{j}' for j in range(n)])

    # Adiciona restrições para cada linha
    for i in range(m):
        coeff = [1]*len(J[i])
        
        # Linha i pode ser coberta pelas colunas em J[i]
        # Toda linha deve ser coberta por pelo menos 1 coluna
        cpx.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=J[i], val=coeff)],
            senses=["G"],
            rhs=[1],
            names=[f'c_{i}']
        )

    # Configurando parâmetros do CPLEX
    cpx.parameters.mip.strategy.heuristicfreq.set(-1)
    cpx.parameters.mip.cuts.mircut.set(-1)
    cpx.parameters.mip.cuts.implied.set(-1)
    cpx.parameters.mip.cuts.gomory.set(-1)
    cpx.parameters.mip.cuts.flowcovers.set(-1)
    cpx.parameters.mip.cuts.pathcut.set(-1)
    cpx.parameters.mip.cuts.liftproj.set(-1)
    cpx.parameters.mip.cuts.zerohalfcut.set(-1)
    cpx.parameters.mip.cuts.cliques.set(-1)
    cpx.parameters.mip.cuts.covers.set(-1)
    cpx.parameters.threads.set(1)
    cpx.parameters.clocktype.set(1)
    cpx.parameters.timelimit.set(1800)

    # Escreve solução em um arquivo
    cpx.write("setcover.lp")
    
    if not display_output:
        cpx.set_log_stream(None)
        cpx.set_error_stream(None)
        cpx.set_warning_stream(None)
        cpx.set_results_stream(None)

    cpx.solve()

    # Output da solução
    status = cpx.solution.get_status()
    progress = cpx.solution.progress.get_num_nodes_processed()
    solution_values = cpx.solution.get_values()
    objective_value = cpx.solution.get_objective_value()

    if display_output:
        print('Solution status:                   %d' % status)
        print('Nodes processed:                   %d' %
                progress)
        tol = cpx.parameters.mip.tolerances.integrality.get()
        print('Optimal value:                     %f' %
            objective_value)
        values = cpx.solution.get_values()
        
    # for y in x:
    #     if values[x[y]] >= 1 - tol:
    #         print("x_" + str(x[y]) + "= " + str(values[x[y]]))

    return status, solution_values, objective_value

In [ ]:
status, solution_values, targetFn_value = modelo_MIP(instance_name='scp41', display_output=False)

In [ ]:
print(f"Colunas utilizadas na solução: {solution_values.count(1)}")

## Modelo com Relaxação Linear

In [5]:
def modelo_LP(instance_name: str, display_output: bool = True):
    """"
    Modelo de relaxação linear para o problema de recobrimento de conjuntos.

    Recebe uma instância no formato encontrado na ORLib e
    implementa uma solução gulosa sem relaxações.
    """

    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize)

    # Obtém e processa os dados da instância
    # m = # linhas
    # n = # colunas
    # C = lista de custos para cada coluna
    # J = conjunto de comprimento m de listas com as colunas que cobrem cada linha (J[i])

    get_data(instance_url=f'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/{instance_name}.txt')
    m, n, C, J = parse_instance(instance_name=instance_name)

    if not m:
        print('Instance parsing failed.')
        return

    # Função objetivo, minimizar os custos (C)
    cpx.variables.add(obj=C,
                        lb=[0] * n, ub=[1] * n,
                        types=[cpx.variables.type.continuous] * n,
                        names=[f'x_{j}' for j in range(n)])

    # Adiciona restrições para cada linha
    for i in range(m):
        coeff = [1]*len(J[i])
        
        # Linha i pode ser coberta pelas colunas em J[i]
        # Toda linha deve ser coberta por pelo menos 1 coluna
        cpx.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=J[i], val=coeff)],
            senses=["G"],
            rhs=[1],
            names=[f'c_{i}']
        )

    # Configurando parâmetros do CPLEX
    cpx.parameters.mip.strategy.heuristicfreq.set(-1)
    cpx.parameters.mip.cuts.mircut.set(-1)
    cpx.parameters.mip.cuts.implied.set(-1)
    cpx.parameters.mip.cuts.gomory.set(-1)
    cpx.parameters.mip.cuts.flowcovers.set(-1)
    cpx.parameters.mip.cuts.pathcut.set(-1)
    cpx.parameters.mip.cuts.liftproj.set(-1)
    cpx.parameters.mip.cuts.zerohalfcut.set(-1)
    cpx.parameters.mip.cuts.cliques.set(-1)
    cpx.parameters.mip.cuts.covers.set(-1)
    cpx.parameters.threads.set(1)
    cpx.parameters.clocktype.set(1)
    cpx.parameters.timelimit.set(1800)

    # Oculta output stream do CPLEX
    if not display_output:
        cpx.set_log_stream(None)
        cpx.set_error_stream(None)
        cpx.set_warning_stream(None)
        cpx.set_results_stream(None)
    
    # CPLEX solver
    cpx.solve()

    # Captura outputs da solução obtida
    status = cpx.solution.get_status()
    progress = cpx.solution.progress.get_num_nodes_processed()
    solution_values = cpx.solution.get_values()
    objective_value = cpx.solution.get_objective_value()
    
    if display_output:
        cpx.write("setcover_LP.lp")
        print('Solution status:                   %d' % status)
        print('Nodes processed:                   %d' %
                progress)
        tol = cpx.parameters.mip.tolerances.integrality.get()
        print('Optimal value:                     %f' %
            objective_value)
    values = cpx.solution.get_values()
    # for y in x:
    #     if values[x[y]] >= 1 - tol:
    #         print("x_" + str(x[y]) + "= " + str(values[x[y]]))

    return status, solution_values, objective_value
    

In [ ]:
lp_solution, lp_solution_values, lp_targetFn_value = modelo_LP(instance_name="scp41", display_output=True)

### Valores médios para classes de benchmark

Baseado nos resultados obtidos por Umetani e Yagiura (2007), realizamos múltiplos testes nas classes de benchmark para fins de comparação.

In [7]:
def benchmark_avg_LP(instance_class: str = "4", display_output=True):
    """
    Esta classe realiza chamadas para o solver com modelo de relaxação linear
    em cada instância da classe passada.

    Retorna:
    - resultado da relaxação médio entre instâncias
    - vetor com resultados das relaxações
    """

    # Classes de instancias disponíveis
    available = {'4', '5', '6', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}
    if instance_class not in available:
        print("Esta classe de isntancias não foi encontrada. Classes disponíveis:")
        print(available)
        return None

    # Ajusta quantidade de instancias por classe
    if instance_class == "4" or instance_class == "5":
        class_size = 10
    else:
        class_size = 5

    # Calcula valor da relaxação e armazena
    targetFn_values = []
    for i in range(1,class_size+1):
        _,_, target_val = modelo_LP(
            instance_name=f'scp{instance_class}{i}', 
            display_output=display_output
        )
        targetFn_values.append(target_val)

        if display_output:
            print("========="*10) 
            print() # Linha vazia entre saídas
            print("========="*10)

    return np.average(np.array(targetFn_values)), targetFn_values

In [ ]:
class_avg, lr_values = benchmark_avg_LP(instance_class="B", display_output=False)

In [13]:
def benchmark_batch(classes: dict, display_output=False):
    """
    Evoca a função de benchmark para múltiplas classes

    Retorna:
    - Dict atualizado para cada classe com tupla resultante
    """

    for c in classes.keys():
        classes[c] = (benchmark_avg_LP(instance_class=c, display_output=False))

    return classes

In [ ]:
results = benchmark_batch(classes={'A':-1, 'B':-1, 'C':-1})

In [ ]:
results

Resultados condizentes com os encontrados nos benchmarks!

## Modelo com Relaxação Lagrangeana

In [8]:
class Lagrangian_SPC():
    """
    Classe que implementa métodos para o modelo de Relaxação Lagrangiana.
    """

    def __init__(self, instance_name: str, ub=None, greedy_heuristic=True) -> None:
        self.instance_name = instance_name
        self.m, self.n, self.costs, self.J = parse_instance(instance_name)
        self.A = self.set_covering_matrix()
        self.hist_lb = None
        self.ub = (np.dot(self.costs, np.ones(self.n))) if ub is None else ub
        self.g_heur = greedy_heuristic
        self.heuristic_J = set()
        self.hist_ub = None
    
    def set_covering_matrix(self):
        """
        Retorna uma matriz de cobertura com dimentsões m x n, valores binários.
        """

        self.A = np.zeros((self.m, self.n), dtype=int)
        for i, cols in enumerate(self.J):
            for j in cols:
                self.A[i][j] = 1
        self.A = self.A.tolist()

        return  self.A
    
    def lagrangian_relaxation(self, u):
        """
        Resolve a relaxação lagrangiana para o problema de recobrimento

        Retorna:
        z_lr (float): Lagrangian relaxation objective value.
        x (list): Solution vector for LR(u).
        """

        # Calcula os custos reduzidos c'_j(u) = c_j - sum(a_{ij} * u_i)
        reduced_costs = np.array(self.costs) - np.dot(u, self.A)

        x = np.zeros(self.n)
        for j in range(self.n):
            if reduced_costs[j] < 0:
                x[j] = 1

        # Computa o subgradiente e a Relaxação Lagrangeana
        subgrad = np.ones(self.m) - np.dot(self.A, x)
        z_lr = np.dot(self.costs, x) + np.dot(u, subgrad)

        return z_lr, x, subgrad
    
    def solve_lagrangian_dual(self, max_iters: int=1000, epsilon=0.02, pi=2.0, plot=False, tolerance=0, log=False):
        """
        Solves the Lagrangian Dual problem to find the best Lagrangian multipliers u.

        Returns:
        best_u (list): Optimal Lagrangian multipliers.
        best_z (float): Optimal Lagrangian objective value.
        best_x (list): Best solution vector.
        """

        # Inicializa multiplicadores lagrangeanos como zeros
        u = np.zeros(self.m)

        best_u = u.copy()
        best_z = -np.inf
        best_x = None

        # Número de iterações sem atualizar o melhor limite inferior
        no_update = 0

        # Critério para reduzir passo T através de pi/2
        half = max_iters // 20
        halved = 0
        
        # Armazena todos os valores de lb explorados
        if plot:
            self.hist_lb = []
        self.hist_ub = []

        # Critério de parada 1: max iterações
        for k in range(max_iters):
            if log:
                print(f"Iteration k = {k}, starting...")

            if no_update == half:
                pi /= 2.0
                no_update = 0
                halved += 1
                
                # Critério de parada 2: pi muito pequeno
                if pi < tolerance:
                    break
                
                if log:
                    print(f"No update limit reached.\nNew value for pi = {pi}. Tolerance = {tolerance}")
                    print(f"Halved {halved} times.")

            # Solve the Lagrangian relaxation for the current u
            z_lr, x, subgrad = self.lagrangian_relaxation(u)

            if log:
                print(f"Current lower bound: {z_lr}")
            if plot:
                self.hist_lb.append(z_lr)
                
            # Update the best solution found so far
            if z_lr > best_z:
                best_z = z_lr
                best_u = u.copy()
                best_x = x.copy()
                no_update = 0

                self.ub = self.__greedy_heuristic2(x, log) if self.g_heur else self.ub
                self.hist_ub.append(self.ub)

                if log:
                    print(f"New best lower bound: {best_z}")
                    print(f"Current upper bound at: {self.ub}")
            else:
                no_update += 1
            
            # Critério de parada 3: solução ótima alcançada
            if self.ub - z_lr < 1:
                break
                
            # Ajusta os multiplicadores de Lagrange com base no subgradiente e lb obtidos
            T = pi * (self.ub - z_lr) / np.sum(np.square(subgrad))
            u = u + (1 + epsilon) * T * subgrad
            u = np.maximum(u, 0)  
        
        if plot:
            _,_,opt = modelo_MIP(instance_name=self.instance_name, display_output=False)
            _,_,lr = modelo_LP(instance_name=self.instance_name, display_output=False)
            self.__plot(opt=opt, l_relax=lr)

        return best_u, best_z, best_x
    
    def __greedy_heuristic(self, x, log=False):
        """
        Heurística gulosa aplicada ao Algoritmo de Subgradiente

        A cada iteração k do algoritmo em que lb é atualizado:
        - Calcula p : min( c_p / |S_p \ j in J*| )
        - J* U {p}

        Quando S_J* = S, retorna ub = \sum c_j , j in J*
        """
        uncovered_rows = set(range(self.m))
        # Conjunto de linhas cobertas por x obtido com informarção dual
        dual_J = set(np.where(x == 1)[0].tolist())

        if log:
            print(f"Current Jk: {dual_J}")
            print(f"Current uncovered rows: {uncovered_rows}")

        while bool(uncovered_rows):
            # Find the column p that minimizes the ratio c_p / |S_p \ (U S_j for j in J*)|
            best_ratio = float('inf')
            best_column = None

            for p in dual_J:
                if p in self.heuristic_J:
                    continue

                # Set of rows covered by column p that are not yet covered by J*
                covered_by_p = set(i for i in range(self.m) if self.A[i][p] == 1)
                new_rows_by_p = covered_by_p & uncovered_rows

                if log:
                    print(f"Column {p} new covered rows: {new_rows_by_p}")

                # If column p covers no new rows, skip it
                if not bool(new_rows_by_p):
                    continue
                
                # Calculate the ratio c_p / |covered_by_p|
                ratio = self.costs[p] / len(covered_by_p)
                
                # If this column provides the best ratio, update the selection
                if ratio < best_ratio:
                    best_ratio = ratio
                    best_column = p

                    if log:
                        print(f"New best ratio found by column {p}: {ratio}")

            # O conjunto J^k foi esgotado e não foi encontrada uma solução viável ainda
            # passamos a usar J\J^k (primal) ao invés de J^k (dual) para completar a heurística
            if (best_column is None) and (bool(uncovered_rows) == True):
                new_J = set(range(self.n))
                # print(new_J)
                new_J -= dual_J
                # print(new_J)
                dual_J = new_J.copy()

                if log:
                    print("J^k não cobre todas as linhas, operando em J\J^k...")
                continue
            
            # Add the selected column p to J*
            self.heuristic_J.add(best_column)
            
            # Remove the rows covered by p from the uncovered set
            uncovered_rows -= set(i for i in uncovered_rows if self.A[i][best_column] == 1)

            if log:
                print(f"Current uncovered rows: {uncovered_rows}")
            
        # Calculate the upper bound as the sum of the costs of the selected columns
        ub_star = sum(self.costs[j] for j in self.heuristic_J)
            
        return ub_star
    
    def __greedy_heuristic2(self, x, log=False):
        """
        Optimized Greedy Heuristic for Subgradient Algorithm.
        """
        uncovered_rows = set(range(self.m))
        # Conjunto de colunas onde x[p] == 1
        dual_J = set(np.where(x == 1)[0].tolist())

        if log:
            print(f"Current Jk: {dual_J}")
            print(f"Current uncovered rows: {uncovered_rows}")

        # Cached list of row coverage for each column
        row_coverage_by_column = [set(i for i in range(self.m) if self.A[i][p] == 1) for p in range(self.n)]
        
        while uncovered_rows:
            best_ratio = float('inf')
            best_column = None

            # Iterate over all columns in dual_J
            for p in dual_J:
                if p in self.heuristic_J:
                    continue

                # Only compute uncovered rows directly
                new_rows_by_p = row_coverage_by_column[p] & uncovered_rows

                if log:
                    print(f"Column {p} new covered rows: {new_rows_by_p}")

                # Skip columns that do not cover any new rows
                if not new_rows_by_p:
                    continue

                # Calculate ratio only if there are uncovered rows covered by p
                ratio = self.costs[p] / len(new_rows_by_p)

                if ratio < best_ratio:
                    best_ratio = ratio
                    best_column = p

                    if log:
                        print(f"New best ratio found by column {p}: {ratio}")

            # If no viable column was found in the current dual_J, switch to the primal set
            if best_column is None and uncovered_rows:
                # Switch to all columns minus those in dual_J (primal set)
                dual_J = set(range(self.n)) - self.heuristic_J

                if log:
                    print("Switching to primal set J \\ J^k...")
                continue

            # Add the selected column to heuristic_J
            if best_column is not None:
                self.heuristic_J.add(best_column)
                uncovered_rows -= row_coverage_by_column[best_column]

                if log:
                    print(f"Selected column {best_column} and updated uncovered rows: {uncovered_rows}")

        # Compute the upper bound
        ub_star = sum(self.costs[j] for j in self.heuristic_J)

        return ub_star

    def __plot(self, opt, l_relax):
        # %matplotlib qt
        fig = plt.figure(figsize=(16,9))
        
        iterations = range(1, len(self.hist_lb) + 1)

        plt.plot(iterations, self.hist_ub, label='Upper Bound $ub^{h}$')
        plt.plot(iterations, self.hist_lb, label='Lower Bound $z_{LR}(u)$')
        plt.axhline(y=opt, linestyle='--', color='red', label='IP Optimal')
        plt.axhline(y=l_relax, linestyle='--', color='magenta', label='Lagrangean Relaxation')

        plt.title("Lagrangian Relaxation: Lower Bound over Iterations", fontsize=16)
        plt.xlabel("Iteration", fontsize=14)
        plt.ylabel("Lower Bound $z_{LR}(u)$", fontsize=14)
        plt.grid(True)
        plt.legend(loc='best')
        
        plt.show()
        

In [ ]:
_,_,opt = modelo_MIP(instance_name='scp41', display_output=False)

In [ ]:
lag_scp = Lagrangian_SPC( instance_name='scp41', ub=opt, greedy_heuristic=True)

u, z, x = lag_scp.solve_lagrangian_dual(plot=False, tolerance=0, log=True)

In [ ]:
# uncovered_rows = set(range(lag_scp.m))
# covered_by_p = set(i for i in range(lag_scp.m) if lag_scp.A[i][12] == 1)
# print(uncovered_rows & covered_by_p)
# uncovered_rows -= covered_by_p
# val = bool(uncovered_rows & covered_by_p)
# print(val)
# if val:
#     print(val)


dual_J = set(np.where(x == 1)[0].tolist())
    

In [ ]:
z